In [1]:
import xarray
from dask.distributed import Client
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import xarray
import pandas as pd
from datetime import datetime, timedelta
import cartopy.crs as ccrs

from preprocess.sza import solarzenithangle


In [2]:

client = Client(n_workers = 12)

In [3]:
hres = xarray.open_zarr('/scratch/snx3000/acarpent/EumetsatData/SEVIRI_WGS_2016-2022_RSS.zarr')
hres

<xarray.Dataset>
Dimensions:      (time: 222290, y: 658, x: 736)
Coordinates:
  * time         (time) datetime64[ns] 2016-01-01 ... 2022-12-31T23:45:00
  * x            (x) float64 -7.775 -7.725 -7.675 -7.625 ... 28.88 28.93 28.98
  * y            (y) float64 28.98 29.03 29.08 29.12 ... 61.68 61.73 61.78 61.83
Data variables:
    IR_016       (time, y, x) float32 dask.array<chunksize=(24, 658, 736), meta=np.ndarray>
    IR_039       (time, y, x) float32 dask.array<chunksize=(24, 658, 736), meta=np.ndarray>
    IR_087       (time, y, x) float32 dask.array<chunksize=(24, 658, 736), meta=np.ndarray>
    IR_097       (time, y, x) float32 dask.array<chunksize=(24, 658, 736), meta=np.ndarray>
    IR_108       (time, y, x) float32 dask.array<chunksize=(24, 658, 736), meta=np.ndarray>
    IR_120       (time, y, x) float32 dask.array<chunksize=(24, 658, 736), meta=np.ndarray>
    IR_134       (time, y, x) float32 dask.array<chunksize=(24, 658, 736), meta=np.ndarray>
    VIS006       (time, y, x) float32 dask.array<chunksize=(24, 658, 736), meta=np.ndarray>
    VIS008       (time, y, x) float32 dask.array<chunksize=(24, 658, 736), meta=np.ndarray>
    WV_062       (time, y, x) float32 dask.array<chunksize=(24, 658, 736), meta=np.ndarray>
    WV_073       (time, y, x) float32 dask.array<chunksize=(24, 658, 736), meta=np.ndarray>
    spatial_ref  int64 ...

In [4]:
sarah = xarray.open_zarr('/scratch/snx3000/kschuurm/DATA/SARAH3.zarr')
sarah

<xarray.Dataset>
Dimensions:   (time: 35040, lat: 503, lon: 801, bnds: 2)
Coordinates:
  * lat       (lat) float32 35.03 35.08 35.13 35.19 ... 61.81 61.86 61.91 61.97
  * lon       (lon) float32 -14.97 -14.92 -14.87 -14.81 ... 27.87 27.92 27.97
  * time      (time) datetime64[ns] 2014-01-01 ... 2015-12-31T23:30:00
Dimensions without coordinates: bnds
Data variables:
    CAL       (time, lat, lon) float32 dask.array<chunksize=(48, 503, 801), meta=np.ndarray>
    SID       (time, lat, lon) float32 dask.array<chunksize=(48, 503, 801), meta=np.ndarray>
    SIS       (time, lat, lon) float32 dask.array<chunksize=(60, 503, 801), meta=np.ndarray>
    lat_bnds  (time, lat, bnds) float32 dask.array<chunksize=(48, 503, 2), meta=np.ndarray>
    lon_bnds  (time, lon, bnds) float32 dask.array<chunksize=(48, 801, 2), meta=np.ndarray>

In [ ]:

lat = hres.lat.copy()
lon = hres.lon.copy()
# del hres

def reindex_sarah3(df):
    return df.reindex(lat=lat, lon=lon, method='nearest').drop_vars(['record_status'])


In [ ]:
fns = glob('/scratch/snx3000/kschuurm/DATA/SARAH3/ORD53913/SIDin*.nc')
i_step = 100

for i in tqdm(range(0, len(fns), i_step)):
    
    sl = slice(i, i+i_step)
    
    sid = xarray.open_mfdataset(fns[sl], 
                            parallel=True,
                            engine='h5netcdf',
                            chunks={'time':60, 'lat':-1, 'lon':-1},
                            preprocess=reindex_sarah3)
    if i == 0:
        sid.to_zarr('/scratch/snx3000/kschuurm/DATA/SARAH3.zarr', mode='w')
    else:
        sid.to_zarr('/scratch/snx3000/kschuurm/DATA/SARAH3.zarr', append_dim='time')


In [ ]:
sis = xarray.open_mfdataset('/scratch/snx3000/kschuurm/DATA/SARAH3/SIS_*.nc', 
                            engine='h5netcdf',
                           parallel=True,
                            chunks={'time':60, 'lat':-1, 'lon':-1},
                           preprocess=reindex_sarah3)
sarah = sis.SIS.to_zarr('/scratch/snx3000/kschuurm/DATA/SARAH3.zarr', mode='a')

In [31]:
fns = glob('/scratch/snx3000/kschuurm/DATA/SARAH3/ORD52533/CALin2014*.nc') + glob('/scratch/snx3000/kschuurm/DATA/SARAH3/ORD52533/CALin2015*.nc')
# print(len(fns))
cal = xarray.open_mfdataset(fns, 
                            parallel=True,
                            engine='h5netcdf',
                            chunks={'time':60, 'lat':-1, 'lon':-1},
                            preprocess=reindex_sarah3)

cal.CAL.to_zarr('/scratch/snx3000/kschuurm/DATA/SARAH3.zarr', mode='a')




In [ ]:
cal = xarray.open_mfdataset(fns, 
                            parallel=True,
                            engine='h5netcdf',
                            chunks={'time':60, 'lat':-1, 'lon':-1},
                            preprocess=reindex_sarah3)
cal.to_zarr('/scratch/snx3000/kschuurm/DATA/SARAH3.zarr',  append_dim='time')

In [33]:

def solarzenithangle_latlon(ds):
    subsample_int = 4
    
    da_temp = ds.channel_1
    
    a = [x for x in range(0, len(da_temp.time)-1, 1)]
    a.append(len(da_temp.time)-1)
    datetimes = pd.to_datetime(da_temp.time[a])
    
    lats = np.arange(da_temp.lat.min(), da_temp.lat.max()+1, 1, dtype=np.float32)
    lons = np.arange(da_temp.lon.min(), da_temp.lon.max()+1, 1, dtype=np.float32)
    
    
    da_sza = xarray.DataArray(coords={'time':datetimes, 'lat':lats, 'lon':lons,},
                          data=np.zeros(shape=(len(datetimes), len(lats), len(lons)),
                                       dtype=np.float16))
    da_sza.name = 'SZA'
    da_sza.attrs.update({'long_name': 'Solar Zenith Angle at sea level',
                      'standard_name': 'solar_zenith_angle',
                      'units':'rad'})
    da_azi = xarray.DataArray(coords={'time':datetimes, 'lat':lats, 'lon':lons,},
                          data=np.zeros(shape=(len(datetimes), len(lats), len(lons)), 
                                        dtype=np.float16))
    da_azi.name = 'AZI'
    da_azi.attrs.update({'long_name': 'Solar Azimuth Angle at sea level',
                      'standard_name': 'solar_azimuth_angle',
                      'units':'rad'})
    
    for i, lat in tqdm(enumerate(lats)):
        for j, lon in enumerate(lons):
            sza, azi = solarzenithangle(datetimes, lat, lon, 0)
            da_sza[:,i, j] = sza
            da_azi[:,i,j] = azi
    
    
    ds = xarray.Dataset({'SZA':da_sza, 'AZI':da_azi})
    return ds


        

In [34]:
ds_sun = solarzenithangle_latlon(hres)

28it [20:49, 44.61s/it]


In [35]:
ds_sun.chunk({'time':1, 'lat':-1, 'lon':-1}).to_zarr('temp.zarr')

/scratch/snx3000/kschuurm/pytorch/lib/python3.9/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 176.65 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [36]:
temp = xarray.open_zarr('temp.zarr')

In [37]:
ds_sun_interp = temp.interp(lat=hres.lat, lon=hres.lon)

In [38]:
temp2 = ds_sun_interp.to_zarr('temp2.zarr')

2024-02-22 17:00:16,070 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-02-22 17:00:20,788 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-02-22 17:00:24,047 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2024-02-22 17:00:28,228 - distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
2024-02-22 17:00:32,727 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
/scratch/snx3000/kschuurm/pytorch/lib/python3.9/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 32.23 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
2024-02-22 17:00:38,796 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time

In [ ]:
temp2 = xarray.open_zarr('temp2.zarr')

In [ ]:
for var in temp2:
    del temp2[var].encoding['chunk']

ds_sun_final = temp2.chunk({'time':60, 'lat':-1, 'lon':-1})

In [ ]:
ds_sun_final.to_zarr('/scratch/snx3000/kschuurm/DATA/HRSEVIRI_corrected.zarr', mode='a')
